In [1]:
library(ggplot2)
library(reshape2)
library(mvtnorm)

In [93]:
# loading data
train<-read.csv("Task1E_train.csv")
test<-read.csv("Task1E_test.csv")
train.data <- train[,-3]
train.label <- train[,3]
test.data <- test[,-3]
test.label <- test[,3]

# length of training and testing data
train.len <- nrow(train.data)
test.len <- nrow(test.data)

7.I

Bayesian Classification

In [94]:
# Bayesian Classifier function
# c0 and c1 are the types of labels
bf <- function(train.data,train.label,test.data,test.label,c0,c1){
    
    # estimated probability of each label type
    p0.hat <- sum(train.label==c0) / nrow(train.data)
    p1.hat <- sum(train.label==c1) / nrow(train.data)
    
    # estimated mean of each label type
    # colMeans find the mean of each column
    mu0.hat <- colMeans(train.data[train.label==c0,])
    mu1.hat <- colMeans(train.data[train.label==c1,])
    
    # counting the number of each label in subset
    # because for example in th efirst 5 rows of training date, there is only one -1 label
    # hence, manually setting the variance to 0, else it will return NaN
    count1<-0
    count2<-0
    for (i in train.label){
        if (i==1){
            count1<-count1+1
                }
        if(i==-1){
            count2<-count2+1
                }}

    if (count1 <= 1){
        sigma0.hat<-0}
    
    if (count1 > 1){
        sigma0.hat <- var(train.data[train.label==c0,])}

    
    if (count2 <= 1){
        sigma1.hat<-0}
    
    if (count2 > 1){
        sigma1.hat <- var(train.data[train.label==c1,])}

                        
    # covariance matrix
    sigma.hat <- p0.hat * sigma0.hat + p1.hat * sigma1.hat
                        
    # posterior values of each class according to naive bayes
    posterior0 <- p0.hat*dmvnorm(x=test.data, mean=mu0.hat, sigma=sigma.hat)
    posterior1 <- p1.hat*dmvnorm(x=test.data, mean=mu1.hat, sigma=sigma.hat)
                        
    # predicting the label from the concept of naive bayes
    test.predict <- ifelse(posterior0 > posterior1, c0, c1)
                        
    # misclassification error
    test.error <- sum(test.label!=test.predict)/nrow(train.data) *100
    
    # error
    return(test.error)
}

In [118]:
# creating empty data frame for errors
error <- data.frame()
# index position of the error dataframe
indx = 1

for (i in 1:500){
    # initializing the current index position of each subset which is 1
    j<-1
    
    # break if 500^th index reached
    if (i==500){
        break
    }
    
    # for the first five data point
    if (i==1){        
        i<-i+4
        # USING THE BC MODEL ON TRAIN AND TEST SETS
        re_model_train<-bf(train.data[j:i,],train.label[j:i],train.data[j:i,],train.label[j:i],1,-1)
        re_model_test<-bf(train.data[j:i,],train.label[j:i],test.data[j:i,],test.label[j:i],1,-1)
        # STORING THE TRAINING AND TESTING ERRORS
        error[indx,'size']<-nrow(train.data[j:i,])
        error[indx,'train_error']<-re_model_train
        error[indx,'test_error']<-re_model_test
        indx<-indx+1
        next
  }
        
    # for indexes other than 5,10,15,25,.....,455
    if (i%%5!=0){
        next
    }


    else{
    
        i<-i+5    
        # USING THE BC MODEL ON TRAIN AND TEST SETS
        re_model_train<-bf(train.data[j:i,],train.label[j:i],train.data[j:i,],train.label[j:i],1,-1)
        re_model_test<-bf(train.data[j:i,],train.label[j:i],test.data[j:i,],test.label[j:i],1,-1)
        # STORING THE TRAINING AND TESTING ERRORS
        error[indx,'size']<-nrow(train.data[j:i,])
        error[indx,'train_error']<-re_model_train
        error[indx,'test_error']<-re_model_test
        indx<-indx+1
    }
    
}

Logistic Regression

In [78]:
# auxiliary function that predicts class labels
predict <- function(w, X, c0, c1){
    sig <- sigmoid(w, X)
    return(ifelse(sig>0.5, c0,c1))
}
    
# auxiliary function that calculate a cost function
cost <- function (w, X, T, c0){
    sig <- sigmoid(w, X)
    return(sum(ifelse(T==1, 1-sig, sig)))
}
# Sigmoid function (=p(C1|X))
sigmoid <- function(w, x){
    return(1.0/(1.0+exp(-w%*%t(cbind(1,x)))))    
}
log <- function(train.data,train.label,tau.max=1000,eta=0.01,epsilon=0.01){
# Initializations
tau.max <- 1000 # maximum number of iterations
eta <- 0.01 # learning rate
epsilon <- 0.01 # a threshold on the cost (to terminate the process)
tau <- 1 # iteration counter
terminate <- FALSE

## Just a few name/type conversion to make the rest of the code easy to follow
X <- as.matrix(train.data) # rename just for conviniance
T <- ifelse(train.label==1,1,-1) # rename just for conviniance

W <- matrix(,nrow=tau.max, ncol=(ncol(X)+1)) # to be used to store the estimated coefficients
W[1,] <- runif(ncol(W)) # initial weight (any better idea?)

# project data using the sigmoid function (just for convenient)
Y <- sigmoid(W[1,],X)

costs <- data.frame('tau'=1:tau.max)  # to be used to trace the cost in each iteration
costs[1, 'cost'] <- cost(W[1,],X,T, 1)
while(!terminate){
    # check termination criteria:
    terminate <- tau >= tau.max | cost(W[tau,],X,T, 1)<=epsilon
    
    # shuffle data:
    train.index <- sample(1:length(train.data), length(train.data), replace = FALSE)
    X <- X[train.index,]
    T <- T[train.index]
    
    # for each datapoint:
    for (i in 1:length(train.data)){
        # check termination criteria:
        if (tau >= tau.max | cost(W[tau,],X,T, 1) <=epsilon) {terminate<-TRUE;break}
        
        Y <- sigmoid(W[tau,],X)
            
        # Update the weights
        W[(tau+1),] <- W[tau,] - eta * (Y[i]-T[i]) * cbind(1, t(X[i,]))
        
        # record the cost:
        costs[(tau+1), 'cost'] <- cost(W[tau,],X,T, 1)
        
        # update the counter:
        tau <- tau + 1
        
        # decrease learning rate:
        eta = eta * 0.999
    }}
    w <- W[tau,]
    return(w)
}

w <- log(train.data,train.label,tau.max=1000,eta=0.01,epsilon=0.01)
s<-0
for (i in 1:500){
    if(predict(w,train.data,1,-1)[i]!=train.label[i]){
        s<-s+1
    }
}
s/500*100
w
predict(w,train.data,1,-1)[1:5]
train.data[j:i,],train.label[j:i],train.data[j:i,],train.label[j:i],1,-1

[1] 30.4

[1] -2.587916  3.333634 -9.901240

[1] -1  1  1 -1 -1

In [129]:
# creating empty data frame for errors
error_lr <- data.frame()
# index position of the error dataframe
indx = 1

for (i in 1:500){
    # initializing the current index position of each subset which is 1
    j<-1
    
    # break if 500^th index reached
    if (i==500){
        break
    }
    
    # for the first five data point
    if (i==1){        
        i<-i+4
        # USING THE BC MODEL ON TRAIN AND TEST SETS
        w <- log(train.data[j:i,],train.label[j:i],tau.max=1000,eta=0.01,epsilon=0.01)
        pred_train <- (predict(w,train.data,1,-1)[j:i])
        train.error <- sum(test.label[j:i]!=pred_train)/nrow(train.data[j:i,]) *100
        pred_test <- (predict(w,test.data,1,-1)[j:i])
        test.error <- sum(test.label[j:i]!=pred_test)/nrow(train.data[j:i,]) *100

#         STORING THE TRAINING AND TESTING ERRORS
        error_lr[indx,'size']<-nrow(train.data[j:i,])
        error_lr[indx,'train_error']<-train.error
        error_lr[indx,'test_error']<-test.error
        indx<-indx+1
        next
  }
        
    # for indexes other than 5,10,15,25,.....,455
    if (i%%5!=0){
        next
    }


    else{
    
        i<-i+5    
        w <- log(train.data[j:i,],train.label[j:i],tau.max=1000,eta=0.01,epsilon=0.01)
        pred_train <- (predict(w,train.data,1,-1)[j:i])
        train.error <- sum(test.label[j:i]!=pred_train)/nrow(train.data[j:i,]) *100
        pred_test <- (predict(w,test.data,1,-1)[j:i])
        test.error <- sum(test.label[j:i]!=pred_test)/nrow(train.data[j:i,]) *100

#         STORING THE TRAINING AND TESTING ERRORS
        error_lr[indx,'size']<-nrow(train.data[j:i,])
        error_lr[indx,'train_error']<-train.error
        error_lr[indx,'test_error']<-test.error
        indx<-indx+1
    }
    
}